# GRIDFINDER WORKSHOP

This notebook will guide you through the use of the _gridfinder_ model to create predictions on the location of medium-voltage grid lines in a given country, based on night-time lights data and OpenStreetMap road networks. This version has been simplified for the workshop.

For more information, please see:
- https://github.com/carderne/gridfinder
- http://blogs.worldbank.org/energy/using-night-lights-map-electrical-grid-infrastructure
- https://engineering.fb.com/connectivity/electrical-grid-mapping/

# 1. Data Preparation
## 1.a. Import necessary Modules
If this fails, you may need to install additional modules, or check that gridfinder is in your path.

In [ ]:
import os
from pathlib import Path

import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation as animation
import seaborn as sns
from IPython.display import display, Markdown

import numpy as np
import rasterio
from rasterstats import zonal_stats
import geopandas as gpd
import folium
import branca

import gridfinder as gf
from gridfinder import save_raster

## 1.b. Set folders and parameters
These are the basic parameters and file names used in the model.

In [ ]:
# The country name here must match the names used in the "data" folder.
country = "kenya"

In [ ]:
# You shouldn't need to change these
data = Path("data")
aoi_in = data / f"{country}.gpkg"
roads_in = data / "roads.gpkg"
ntl_in = data / "ntl.tif"

In [ ]:
# Do not need to be changed
outputs = Path("outputs")
ntl_out = outputs / "ntl_clipped.tif"
ntl_thresh_out = outputs / "ntl_thresh.tif"
targets_out = outputs / "targets.tif"
roads_out = outputs / "roads.tif"

dist_out = outputs / "dist.tif"
guess_out = outputs / "guess.tif"
guess_skeletonized_out = outputs / "guess_skel.tif"
guess_nulled = outputs / "guess_nulled.tif"
guess_vec_out = outputs / "guess.gpkg"
animate_out = outputs / "animated"

In [ ]:
# These parameters control how the model operates.
ntl_threshold = 0.1  # threshold when converting filtered NTL to binary (probably shouldn't change)
upsample_by = 1      # factor by which to upsample before processing roads (both dimensions are scaled by this)
cutoff = 0.0         # cutoff to apply to output dist raster, values below this are considered grid

# 2. Geoprocessing and Algorithm Tuning
## 2.a. Create Filter
This filter is used to highlight areas of the night-time lights that are significantly brighter that their surroundings.

In [ ]:
ntl_filter = gf.create_filter()

X = np.fromfunction(lambda i, j: i, ntl_filter.shape)
Y = np.fromfunction(lambda i, j: j, ntl_filter.shape)

fig = plt.figure()
sns.set()
ax = fig.gca(projection='3d')
ax.plot_surface(X, Y, ntl_filter, cmap=cm.coolwarm, linewidth=0, antialiased=False)

## 2.b. Clip, filter and resample night-time lights
In this step, we clip the night-time lights, and then use the filter from above to calculate areas that are expected to have electricity access.

In [ ]:
aoi = gpd.read_file(aoi_in)
aoi_diss = aoi.dissolve(by="admin")
buff = aoi_diss.copy()
buff.geometry = aoi_diss.buffer(0.1)

# Clip NTL raster
ntl_clipped, affine, crs = gf.clip_raster(ntl_in, buff)
gf.save_raster(ntl_out, ntl_clipped, affine, crs, nodata=None)

In [ ]:
ntl_thresh, affine = gf.prepare_ntl(ntl_out,
                                    buff,
                                    ntl_filter=ntl_filter,
                                    threshold=ntl_threshold,
                                    upsample_by=upsample_by)
save_raster(ntl_thresh_out, ntl_thresh, affine)
# Clip to actual AOI
targets, affine, _ = gf.clip_raster(ntl_thresh_out, aoi_diss)
gf.save_raster(targets_out, targets, affine)

print("Targets prepared")
plt.imshow(targets, cmap="viridis")

## 2.c. Roads: assign values, clip and rasterize
We take the raw roads data from OSM and assign different 'costs' to different classes of roads. This means the algorithm will prefer to follow larger roads (motorways) over smaller roads or empty land.

In [ ]:
roads_raster, affine = gf.prepare_roads(roads_in,
                                        aoi_in,
                                        targets_out)
save_raster(roads_out, roads_raster, affine, nodata=-1)
print("Costs prepared")
plt.imshow(roads_raster, cmap='viridis', vmin=0, vmax=1)

## 2.d. Get targets and costs and run algorithm

In [ ]:
targets, costs, start, affine = gf.get_targets_costs(targets_out, roads_out)

In [ ]:
dist = gf.optimise(targets, costs, start,
                   jupyter=True,
                   animate=True,
                   affine=affine,
                   animate_path=animate_out)
save_raster(dist_out, dist, affine)
plt.imshow(dist)

# 3. Visualizing Results
## 3.a. Filter dist results to grid guess
Then from this result, we extract the locations below the given cutoff. These are assumed to be locations that have MV infrastructure.

In [ ]:
guess, affine = gf.threshold(dist_out, cutoff=cutoff)
save_raster(guess_out, guess, affine)

guess_skel, affine = gf.thin(guess_out)
save_raster(guess_skeletonized_out, guess_skel, affine)

print("Got guess and skeletonized")
plt.imshow(guess_skel)

## 3.c. Convert to geometry
To make it easier to work with, we convert this raster result into a vector.

In [ ]:
mv = gf.raster_to_lines(guess_skeletonized_out)
mv.crs = {"init": "epsg:4326"}
mv.to_file(guess_vec_out, driver="GPKG")
print("Converted to geometry")

In [ ]:
minx, miny, maxx, maxy = list(mv.bounds.iloc[0])
bounds = ((miny, minx), (maxy, maxx))

m = folium.Map(control_scale=True)
m.fit_bounds(bounds)
folium.GeoJson(mv).add_to(m)
m

# 4. Applying zonal statitics

In [ ]:
stats = zonal_stats(vectors=aoi, raster=guess_skeletonized_out, affine=affine, stats='sum')
aoi["sum_mv"] = [int(s["sum"]) for s in stats]

In [ ]:
colorscale = branca.colormap.linear.YlOrRd_09.scale(0, aoi["sum_mv"].max())
colorscale.caption = 'Total MV per region'

In [ ]:
m = folium.Map(control_scale=True)
m.fit_bounds(bounds)
def style_function(feature):
    sum_mv = feature["properties"]["sum_mv"]
    return {
        'fillOpacity': 0.5,
        'weight': 1,
        'color': "black",
        'fillColor': colorscale(sum_mv)
    }
folium.GeoJson(
    aoi,
    style_function = style_function
).add_to(m)

m.add_child(colorscale)
m

# 5. Challenge
1. Run the MV Predictions for 3 other countries other than Ghana and generate maps for each.
2. Calculate the estimated population within 1 km of the MV electrical grid prediction.
3. Model Development and Policy Discussion.

In [ ]:
# Load population data
pop_in = data / "ghs_africa.tif"
pop_total = zonal_stats(vectors=aoi_diss, raster=pop_in, stats='sum')[0]["sum"]

In [ ]:
# Buffer by set distance
buff_dist = 1  # in km
buff_dist_deg = buff_dist * 0.01  # approximate
mv_buff = mv.copy()
mv_buff.geometry = mv_buff.buffer(buff_dist_deg)

# Calculate zonal population statistics
pop_mv = zonal_stats(vectors=mv_buff, raster=pop_in, stats='sum')[0]["sum"]

In [ ]:
print(f"Total population:\t{pop_total:.0f}")
print(f"Population within 1km:\t{pop_mv:.0f}")
print(f"Percentage within 1km:\t{100*pop_mv/pop_total:.0f}%")